In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [2]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [4]:
INPUT_DIR = "./output/20240729-165803/artifacts"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATES_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [5]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 453


[2024-07-29T10:00:01Z WARN  lance::dataset] No existing dataset at /Users/dinhuyennhi/untitled folder/output/20240729-165803/artifacts/lancedb/entity_description_embeddings.lance, it will be created


,level,title,type,description,source_id,community,degree,human_readable_id,id,size,graph_embedding,top_level_node_id,x,y
0,0,MỞ TÀI KHOẢN TRỰC TUYẾN,DỊCH VỤ NGÂN HÀNG,Mở tài khoản trực tuyến dành cho Quý Khách hàn...,35308f256e55bb35f3b3f89f01021eb6,1,3,0,b45241d70f0e43fca764df95b2b81f77,3,None,b45241d70f0e43fca764df95b2b81f77,0,0
1,0,QUÝ KHÁCH HÀNG,NGƯỜI,Quý Khách hàng là cá nhân có quốc tịch Việt Na...,35308f256e55bb35f3b3f89f01021eb6,1,1,1,4119fd06010c494caa07f439b333f4c5,1,None,4119fd06010c494caa07f439b333f4c5,0,0
2,0,MBS,CÔNG TY CHỨNG KHOÁN,MBS is a securities company that provides onli...,"345c8f0818dbf83382a52b5cbaed16b1,35308f256e55b...",3,12,2,d3835bf3dda84ead99deadbeac5d0d7d,12,None,d3835bf3dda84ead99deadbeac5d0d7d,0,0
3,0,EKYC,GIẢI PHÁP,"""EKYC (Electronics Know Your Customer – Định d...","243393bf6cd9c9bbab98e416726260d5,35308f256e55b...",1,2,3,077d2820ae1845bcbb1803379a3d1eae,2,None,077d2820ae1845bcbb1803379a3d1eae,0,0
4,0,KHÁCH HÀNG,NGƯỜI,"""KHÁCH HÀNG"" là người có nhu cầu thay đổi hoặc...","243393bf6cd9c9bbab98e416726260d5,345c8f0818dbf...",0,21,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,21,None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,0,0


#### Read relationships

In [6]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 159


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,MỞ TÀI KHOẢN TRỰC TUYẾN,QUÝ KHÁCH HÀNG,1.0,Mở tài khoản trực tuyến dành cho Quý Khách hàn...,[35308f256e55bb35f3b3f89f01021eb6],f2ff8044718648e18acef16dd9a65436,0,3,1,4
1,MỞ TÀI KHOẢN TRỰC TUYẾN,MBS,1.0,MBS xây dựng giải pháp eKYC giúp Khách hàng ho...,[35308f256e55bb35f3b3f89f01021eb6],00d785e7d76b47ec81b508e768d40584,1,3,12,15
2,MỞ TÀI KHOẢN TRỰC TUYẾN,EKYC,1.0,eKYC giúp Khách hàng hoàn tất mọi giao dịch ch...,[35308f256e55bb35f3b3f89f01021eb6],87915637da3e474c9349bd0ae604bd95,2,3,2,5
3,MBS,CHỨNG KHOÁN,1.0,MBS hoạt động trong lĩnh vực chứng khoán,[ea5adc4e47eb4a5cb85381e5d41973bf],8f1eba29f39e411188200bf0d14628ec,3,12,4,16
4,MBS,KHÁCH HÀNG,4.0,"""KHÁCH HÀNG can place orders with MBS to buy o...","[345c8f0818dbf83382a52b5cbaed16b1, 4d2086016ef...",7282c73622b8408e97289d959faff483,4,12,21,33


In [9]:
covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATES_TABLE}.parquet")

claims = read_indexer_covariates(covariate_df)

print(f"Claim records: {len(claims)}")
covariates = {"claims": claims}

Claim records: 98


#### Read community reports

In [10]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 31


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,28,# Customers and Securities Trading Community\n...,2,7.5,Customers and Securities Trading Community,The impact severity rating is high due to the ...,The community revolves around customers engagi...,[{'explanation': 'Customers in this community ...,"{\n ""title"": ""Customers and Securities Trad...",6edc09c7-40f9-4788-9563-3f0dc80948a6
1,29,# MBS Mobile App and Tài Khoản Chứng Khoán Com...,2,7.5,MBS Mobile App and Tài Khoản Chứng Khoán Commu...,The impact severity rating is high due to the ...,The community revolves around the MBS Mobile A...,[{'explanation': 'The MBS Mobile App serves as...,"{\n ""title"": ""MBS Mobile App and T\u00e0i K...",6982fe05-bc9d-4976-ada3-524cd26637db
2,30,# SGDCK and ĐVKD in Stock Market Community\n\n...,2,7.5,SGDCK and ĐVKD in Stock Market Community,The impact severity rating is high due to the ...,"The community consists of SGDCK, a stock marke...",[{'explanation': 'SGDCK plays a crucial role i...,"{\n ""title"": ""SGDCK and \u0110VKD in Stock ...",ae7fce0a-a2ed-4d5d-936e-a1e9c9e4de81
3,10,# TÀI KHOẢN and Financial Transactions\n\nThe ...,1,7.5,TÀI KHOẢN and Financial Transactions,The impact severity rating is high due to the ...,"The community revolves around TÀI KHOẢN, a fin...",[{'explanation': 'TÀI KHOẢN plays a central ro...,"{\n ""title"": ""T\u00c0I KHO\u1ea2N and Finan...",988793ca-7665-4e2b-ad13-85fc7082688b
4,11,# KHÁCH HÀNG and MBS\n\nThe community revolves...,1,7.5,KHÁCH HÀNG and MBS,The impact severity rating is high due to the ...,The community revolves around the entity KHÁCH...,[{'explanation': 'KHÁCH HÀNG plays a central r...,"{\n ""title"": ""KH\u00c1CH H\u00c0NG and MBS""...",9d827d1e-bde4-4108-8da3-826d174650e5


#### Read text units

In [11]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 48


,id,text,n_tokens,document_ids,entity_ids,relationship_ids,covariate_ids
0,35308f256e55bb35f3b3f89f01021eb6,Mở tài khoản trực tuyến\n- Mở tài khoản trực ...,300,[318048eaf88588db1a6df342ae8429ad],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[f2ff8044718648e18acef16dd9a65436, 00d785e7d76...","[9b82b7c4-97f0-491e-a25e-5c5a45471f6f, 35e9709..."
1,f3468129c89327e881e2bf6ca6f24e9e,nh danh khách hàng điện tử) giúp Khách hàng ho...,300,[318048eaf88588db1a6df342ae8429ad],"[077d2820ae1845bcbb1803379a3d1eae, 3671ea0dd4e...","[4f2c665decf242b0bfcaf7350b0e02ed, 66cdf168f36...",[3baba67d-22d8-433a-99a7-ab5d24fdf39b]
2,243393bf6cd9c9bbab98e416726260d5,làm theo các bước hướng dẫn dưới đây:\n+ Bước...,300,[318048eaf88588db1a6df342ae8429ad],"[077d2820ae1845bcbb1803379a3d1eae, 3671ea0dd4e...","[66cdf168f36d4a57a505028c97dc06e0, 896d2a51e8d...","[89a6331e-e1dc-4893-a09f-bb60cb790c3a, e59f6b5..."
3,ea5adc4e47eb4a5cb85381e5d41973bf,"ứng khoán tại MBS, Khách hàng có các lựa chọn ...",300,[318048eaf88588db1a6df342ae8429ad],"[d3835bf3dda84ead99deadbeac5d0d7d, 3671ea0dd4e...","[8f1eba29f39e411188200bf0d14628ec, 896d2a51e8d...","[20c0a117-7f50-4ffa-9b0a-b2913dcfdbf1, 1a0628c..."
4,beefdde86a1e5bb821c4b7b99690494e,(mất phí): Hệ thống tự động gợi ý danh sách t...,300,[318048eaf88588db1a6df342ae8429ad],"[3671ea0dd4e84c1a9b02c5ab2c8f4bac, d91a266f766...","[14555b518e954637b83aa762dc03164e, b1f6164116d...",[2ca703da-9f0b-43af-8bb9-3b302c55ae50]


In [17]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = "gpt-3.5-turbo-0125"
embedding_model = "text-embedding-3-small"

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

### Create local search context builder

In [18]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    covariates=covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

### Create local search engine

In [19]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [20]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

### Run local search on sample queries

In [22]:
question = "MỞ TÀI KHOẢN KIỂU GÌ?"
result = await search_engine.asearch(question)
print(result.response)

Khi mở tài khoản giao dịch chứng khoán, có nhiều cách thức khác nhau mà khách hàng có thể lựa chọn:

1. **Mở Tài Khoản Trực Tuyến:** Quý Khách hàng có thể mở tài khoản trực tuyến thông qua website hoặc ứng dụng di động. Quá trình này thường bao gồm việc đăng ký thông tin cá nhân, xác minh danh tính và hoàn tất các bước mở tài khoản trực tuyến. Sau khi tài khoản được kích hoạt, Quý Khách có thể sử dụng ngay tài khoản để giao dịch chứng khoán qua các kênh trực tuyến hoặc tổng đài.

2. **Mở Tài Khoản Trực Tiếp:** Quý Khách hàng cũng có thể mở tài khoản trực tiếp tại các sàn giao dịch. Quá trình này thường đòi hỏi Quý Khách hàng đến trực tiếp tại điểm giao dịch để hoàn tất thủ tục mở tài khoản.

3. **Mở Tài Khoản qua App:** Nếu Quý Khách hàng đã có tài khoản ngân hàng, có thể mở tài khoản chứng khoán thông qua ứng dụng di động của ngân hàng. Quá trình này thường đơn giản và tiện lợi, giúp Quý Khách hàng dễ dàng tiếp cận dịch vụ giao dịch chứng khoán.

Những cách thức này đều nhằm mục đích 

#### Inspecting the context data used to generate the response

In [23]:
result.context_data["entities"].head()

,id,entity,description,number of relationships,in_context
0,8,TÀI KHOẢN,"""TÀI KHOẢN"" là một dịch vụ tài chính cho phép ...",11,True
1,98,XÁC NHẬN SỐ DƯ TÀI KHOẢN,,1,True
2,145,ĐIỀU KHOẢN SẢN PHẨM,"""Điều khoản sản phẩm là các quy định và điều k...",2,True
3,30,QUẢN LÝ TÀI KHOẢN,"""QUẢN LÝ TÀI KHOẢN"" is a service that allows c...",1,True
4,0,MỞ TÀI KHOẢN TRỰC TUYẾN,Mở tài khoản trực tuyến dành cho Quý Khách hàn...,3,True


In [24]:
result.context_data["relationships"].head()

,id,source,target,description,weight,rank,links,in_context
0,53,TÀI KHOẢN GIAO DỊCH CHỨNG KHOÁN,DỊCH VỤ QUẢN LÝ TÀI KHOẢN,Tài khoản giao dịch chứng khoán được sử dụng đ...,1.0,7,1,True
1,153,SỞ GIAO DỊCH CHỨNG KHOÁN,ĐIỀU KHOẢN SẢN PHẨM,Điều khoản sản phẩm chứng khoán được quy định ...,1.0,4,1,True
2,17,KHÁCH HÀNG,TÀI KHOẢN,"The entity ""KHÁCH HÀNG"" is actively engaging i...",5.0,32,4,True
3,16,KHÁCH HÀNG,TÀI KHOẢN CHỨNG KHOÁN,Khách hàng cần mở tài khoản chứng khoán để gia...,1.0,23,4,True
4,31,KHÁCH HÀNG,CHUYỂN KHOẢN CHỨNG KHOÁN,Chuyển khoản chứng khoán theo yêu cầu của Khác...,2.0,23,4,True


In [25]:
result.context_data["reports"].head()

,id,title,content
0,28,Customers and Securities Trading Community,# Customers and Securities Trading Community\n...
1,7,Tổ Chức Phát Hành and Chứng Khoán,# Tổ Chức Phát Hành and Chứng Khoán\n\nThe com...


In [26]:
result.context_data["sources"].head()

,id,text
0,3,"ứng khoán tại MBS, Khách hàng có các lựa chọn ..."
1,9,tiền theo cụm số khách hàng nhập.\nĐối với Kh...
2,2,làm theo các bước hướng dẫn dưới đây:\n+ Bước...
3,8,ệ thống tự động gợi ý danh sách STK theo thông...
4,7,c trực tiếp tại các sàn giao dịch.\n\nHướng dẫ...


In [27]:
if "claims" in result.context_data:
    print(result.context_data["claims"].head())

Empty DataFrame
Columns: [in_context]
Index: []


### Question Generation

This function takes a list of user queries and generates the next candidate questions.

In [28]:
question_generator = LocalQuestionGen(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
)

In [29]:
question_history = [
    "Mở tài khoản kiểu gì?",
    "Cách mở tài khoản trực tiếp?",
]
candidate_questions = await question_generator.agenerate(
    question_history=question_history, context_data=None, question_count=5
)
print(candidate_questions.response)

['- What are the key entities involved in securities trading activities?', '- How does eKYC impact customer onboarding processes?', '- What is the role of MBS in facilitating customer transactions?', '- What services are provided by DỊCH VỤ GỬI TIỀN in the securities trading community?', '- How does TÀI KHOẢN GIAO DỊCH CHỨNG KHOÁN contribute to stock trading activities?']
